In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo

In [2]:
all_rev = pd.read_csv('type_classification_tokens.csv')
all_rev.drop(['Unnamed: 0'], axis=1, inplace=True)
all_rev

,prod_name,rating,skin_type,review,tokens
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여드름 고민...,"[('제품', 'NNG'), ('은', 'JX'), ('정말', 'MAG'), ('..."
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,간단정리 자극없이 무난함 봄여름가을에 적당한 수분감 겨울엔 두겹 레이어링하면 쫀쫀 ...,"[('간단', 'XR'), ('정리', 'NNG'), ('자극', 'NNG'), (..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,나의 피부상태 복합성 트러블 조금만 자극적이면 피부 뒤집어짐 장점 여드름 짜거나 엄...,"[('나', 'NP'), ('의', 'JKG'), ('피부', 'NNG'), ('상..."
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요 진짜 닥터지 최고입니다 건성분들이 쓰시기에는 너무 빨리 날아가서 피부에 ...,"[('또', 'MAJ'), ('사', 'VV'), ('았', 'EP'), ('어요'..."
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,닥터지 크림은 워낙 유명하기도 하고 성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...,"[('닥터', 'NNP'), ('지', 'NNB'), ('크림', 'NNP'), (..."
...,...,...,...,...,...
90417,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,최애 입니다이제 이것만 쓸거예요또사러 가야지 눈누난나 신난다,"[('최', 'XPN'), ('애', 'NNG'), ('입', 'NNG'), ('니..."
90418,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,자극이 없고 일단은 너무 촉촉해서 좋아요 이것만 쓰게 될것같아,"[('자극', 'NNG'), ('이', 'JKS'), ('없', 'VA'), ('고..."
90419,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,배송도 빠르고 세일할 때 사야할거 같아서 샀어요송악크림 건성한테 정말 좋을거같아요,"[('배', 'NNG'), ('송도', 'NNP'), ('빠르', 'VA'), ('..."
90420,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,가벼운제형이고 끈적거리지 않아서 좋았어요 그리고 진정에도 좋습니다,"[('가볍', 'VA'), ('ㄴ', 'ETM'), ('제', 'XSN'), ('형..."


# func1: 검색어가 제품명에 포함된 제품

In [9]:
def find_name_product(product_name_list, df, mention_df):
    data_list = []
    product_name_list = list(product_name_list)
    product_name = '|'.join(product_name_list)
    candidate = df[df['product_name'].str.contains(product_name)].index
    
    categories = df.columns[1:]
    categories = [*categories, categories[0]]

    for i in candidate:
        
        product1 = df[categories[:len(categories)-1]].loc[i]
        product1 = [*product1, product1[0]]
        
        ##### 언급 비율 -> meta data로 추가 #####
        proportion = mention_df[categories[:len(categories)-1]].loc[i]
        proportion = [*proportion, proportion[0]]
        
        data_list.append(go.Scatterpolar(r=product1, theta=categories, meta=proportion, fill='toself',  name=df['product_name'][i]))
        
    return data_list

# func2: 별점 순 상위 n개의 제품
- 리뷰 수 적어서 속성문장 거의 없는 제품이 상위에 등장
    - [별점 높은 - 리뷰 수 너무 적은 제품은 제외]
    - 평균 5점인 리뷰가 너무 많아서 리뷰 수 t개 이하면 제외

In [4]:
def best_n_product(n, df, best_list, mention_df):
    data_list = []
    
    categories = df.columns[1:]
    categories = [*categories, categories[0]]
    
    for i in range(n):
        idx = df[df['product_name']==best_list[i]].index[0] # 제품 index
        
        product1 = df[categories[:len(categories)-1]].loc[idx]
        product1 = [*product1, product1[0]]
        
        ##### 언급 비율 -> meta data로 추가 #####
        proportion = mention_df[categories[:len(categories)-1]].loc[idx]
        proportion = [*proportion, proportion[0]]
        
        data_list.append(go.Scatterpolar(r=product1, theta=categories, meta=proportion, fill='toself',  name=df['product_name'][idx]))
        
    return data_list

# 그래프 display

In [19]:
# 1. 피부 타입에 맞는 데이터 준비
skin = input('피부 타입을 입력하세요[복합성/지성/건성] : ')

if skin=='복합성':
    # 긍정 비율 csv
    sen_df = pd.read_csv('after_senti_csv/senti_pos_percent_combi.csv')
    sen_df.columns = ['product_name', '피부 진정', '보습 및 수분감', '가성비', '주름 개선', '유분감', '미백 효과']

    # 언급 비율 csv
    mention_df = pd.read_csv('after_senti_csv/senti_aspect_percent_combi.csv')
    mention_df.columns = ['product_name', '피부 진정', '보습 및 수분감', '가성비', '주름 개선', '유분감', '미백 효과']
    
elif skin=='건성':
    sen_df = pd.read_csv('after_senti_csv/senti_pos_percent_dry.csv')
    sen_df.columns = ['product_name', '보습', '화장 전 스킨케어', '가성비', '주름 개선', '피부 진정', '미백 효과']

    mention_df = pd.read_csv('after_senti_csv/senti_aspect_percent_dry.csv')
    mention_df.columns = ['product_name', '보습', '화장 전 스킨케어', '가성비', '주름 개선', '피부 진정', '미백 효과']

elif skin=='지성':
    sen_df = pd.read_csv('after_senti_csv/senti_pos_percent_oily.csv')
    sen_df.columns = ['product_name', '피부 진정', '보습', '가성비', '주름 개선']

    mention_df = pd.read_csv('after_senti_csv/senti_aspect_percent_oily.csv')
    mention_df.columns = ['product_name', '피부 진정', '보습', '가성비', '주름 개선']


# 평균 별점 내림차순 정렬
type_df = all_rev[all_rev['skin_type']==skin]

rating_avg = []

prod_name = set(sen_df['product_name'].values.tolist())
prod_name = list(prod_name)

for prod in prod_name:
    rating_list = type_df[type_df['prod_name']==prod]['rating'].values.tolist()
    
    if len(rating_list) < 11: # 리뷰 수 미달 시 제외해봄
        rating_avg.append(-1)
    
    else:
        rating = float('%0.2f' % (sum(rating_list) / len(rating_list)))
        rating_avg.append(rating)

rating_avg_type = pd.DataFrame({'prod_name':prod_name,
                                 'average':rating_avg})

rating_avg_type.sort_values('average', ascending=False, inplace=True)
rating_avg_type.reset_index(drop=True, inplace=True)


# 2-1 / 2-2
display = input('1. 제품명 검색\n2. 인기순 상위 n개 제품 보기\n: ')

# 2-1. 특정 제품명 입력 (이 때 띄어쓰기 하면 안됨)
if display=='1':
    cream_product = input('제품 명을 입력하세요: ')
    cream_product= cream_product.split(',')
    data = find_name_product(cream_product, sen_df, mention_df)

# 2-2. 별점 평균 상위 n개 제품 확인
elif display=='2':
    topN = input('인기순 n개의 제품: ')
    best_list = rating_avg_type['prod_name'].values.tolist()
    data = best_n_product(5, sen_df, best_list, mention_df)


# 3. 그래프 display
fig = go.Figure(data=data,
               layout=go.Layout(title=go.layout.Title(text='Grade'),
                               polar={'radialaxis': {'visible':True}},
                               showlegend=True))

fig.update_traces(hovertemplate="만족해요: %{r}%"+"<br>키워드: %{theta}"+"<br>언급비율: %{meta}%", selector={'type':'scatterpolar'})
#fig.update_traces(texttemplate=mention , selector={'type':'scatterpolar'})
fig.update_polars(angularaxis_tickfont_size=20)

pyo.plot(fig)  

피부 타입을 입력하세요[복합성/지성/건성] : 건성
1. 제품명 검색
2. 인기순 상위 n개 제품 보기
: 2
인기순 n개의 제품: 5


'temp-plot.html'